In [1]:
!pip install grpcio grpcio-status

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 17.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 KB 33.5 MB/s eta 0:00:00


In [ ]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.remote("sc://0.0.0.0").getOrCreate()